In [15]:
# 1️⃣ Connect to Azure ML workspace
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, automl, Input
from azure.ai.ml.constants import AssetTypes

subscription_id = "SUBSCRIPTION ID"
resource_group = "RESOURCE GROUP"
workspace_name = "WORKPLACE NAME"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace_name
)



In [40]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# Example: using the correct version from Studio
train_input = Input(
    type=AssetTypes.MLTABLE,
    path="azureml:foodpanda_processed:1"  # match exactly
)


In [38]:
assets = ml_client.data.list(name="foodpanda_processed")
for a in assets:
    print(a.name, a.version)


foodpanda_processed 1


In [41]:
from mltable import load
df = load("azureml:foodpanda_processed:1").to_pandas_dataframe()
print(df.head())


Found the config file in: /config.json


  StoreId                  createdAt  \
0    zn0n  2023-12-15T14:51:52+00:00   
1    geap  2023-07-14T14:37:05+00:00   
2    wej9  2024-01-16T11:10:25+00:00   
3    p5qy  2024-01-03T13:07:22+00:00   
4    g2iw  2023-07-09T10:16:59+00:00   

                                                text  isAnonymous  \
0                         Best food ever. Literally!        False   
1  The meat was far INFERIOR to what I expected.....        False   
2  อร่อย หมูกรอบสับชิ้นใหญ่ค่ะ ชอบ แถมให้ซอสซีอิ้...        False   
3  Ordered pad thai without kung heng. In thai. T...        False   
4  ข้าวมันไก่อร่อยมากค่ะ ชอบที่กลิ่นไก่ไม่แรง \nเ...        False   

                                             replies  likeCount  isLiked  \
0                                                 []        0.0    False   
1                                                 []        0.0    False   
2  [{'id': '000228c7-1cae-4309-b796-0de063c39092'...        0.0    False   
3                                       

In [ ]:
from azure.ai.ml import automl, Input
from azure.ai.ml.constants import AssetTypes

train_input = Input(
    type=AssetTypes.MLTABLE,
    path="azureml:foodpanda_processed:1"
)

tab_job = automl.classification(
    compute="cpu-spot-s",
    experiment_name="week2_sentiment_explorer",
    training_data=train_input,
    target_column_name="sentiment",
    primary_metric="accuracy",
    featurization="auto",  # enable featurization
    featurization_settings={
        "datetime_columns": ["createdAt"]  # force AutoML to parse as datetime
    },
    tags={"project": "sentiment_explorer", "modality": "tabular_with_text_and_dates"}
)

tab_job.set_limits(timeout_minutes=90, max_trials=12, max_concurrent_trials=1)

returned_tab_job = ml_client.jobs.create_or_update(tab_job)
print("Job submitted:", returned_tab_job.name)
